In [22]:
import torch
import torch.nn as nn

from torch.nn import init
import torch.nn.functional as F
from torch.autograd import Variable

import time
import random
import numpy as np

from sklearn.metrics import f1_score
from collections import defaultdict

- Simple supervised GraphSAGE model as well as examples running the model on the Cora and Pubmed datasets.

### Cora数据加载

In [9]:
def load_cora():
    num_nodes = 2708
    num_feats = 1433
    feat_data = np.zeros((num_nodes, num_feats))
    labels = np.empty((num_nodes, 1), dtype=np.int64)
    
    node_map = {}
    label_map = {}
    
    with open(r"C:\Users\sss\Desktop\cora/cora.content") as fp:
        for i, line in enumerate(fp):
            info = line.strip().split()
            feat_data[i, :] = list(map(float, info[1: -1]))  # list
            node_map[info[0]] = i
            if not info[-1] in label_map:
                label_map[info[-1]] = len(label_map)
            labels[i] = label_map[info[-1]]
            
    
    adj_lists = defaultdict(set)
    with open(r"C:\Users\sss\Desktop\cora/cora.cites") as fp:
        for i, line in enumerate(fp):
            info = line.strip().split()
            paper1 = node_map[info[0]]
            paper2 = node_map[info[1]]
            adj_lists[paper1].add(paper2)
            adj_lists[paper2].add(paper1)
            
    return feat_data, labels, adj_lists

In [15]:
def load_pubmed():
    # hardcoded for simplicity...
    num_nodes = 19717
    num_feats = 500
    feat_data = np.zeros((num_nodes, num_feats))
    labels = np.empty((num_nodes, 1), dtype=np.int64)
    
    node_map = {}
    
    with open(r"C:\Users\sss\Desktop\pubmed/Pubmed-Diabetes.NODE.paper.tab") as fp:
        fp.readline()
        feat_map = {entry.split(":")[1]: i - 1 for i, entry in enumerate(fp.readline().split("\t"))}
        for i, line in enumerate(fp):
            info = line.split("\t")
            node_map[info[0]] = i
            labels[i] = int(info[1].split("=")[1]) - 1
            for word_info in info[2: -1]:
                word_info = word_info.split("=")
                feat_data[i][feat_map[word_info[0]]] = float(word_info[1])
                
    adj_lists = defaultdict(set)
    with open(r"C:\Users\sss\Desktop\pubmed/Pubmed-Diabetes.DIRECTED.cites.tab") as fp:
        fp.readline()
        fp.readline()
        for line in fp:
            info = line.strip().split("\t")
            paper1 = node_map[info[1].split(":")[1]]
            paper2 = node_map[info[-1].split(":")[1]]
            adj_lists[paper1].add(paper2)
            adj_lists[paper2].add(paper1)
            
    return feat_data, labels, adj_lists

In [16]:
feat_data, labels, adj_lists = load_pubmed()

### Encoder
- Encodes a node's using 'convolutional' GraphSage approach

In [23]:
class Encoder(nn.Module):
    def __init__(self, features, feature_dim, embed_dim, adj_lists, aggregator, num_sample=10, base_model=None, gcn=False, cuda=False, feature_transform=False):
        super(Encoder, self).__init__()
        
        self.features = features
        self.feat_dim = feature_dim
        self.adj_lists = adj_lists
        self.aggregator = aggregator
        self.num_sample = num_sample
        if base_model != None:
            self.base_model = base_model

        self.gcn = gcn
        self.embed_dim = embed_dim
        self.cuda = cuda
        self.aggregator.cuda = cuda
        self.weight = nn.Parameter(
            torch.FloatTensor(embed_dim, self.feat_dim if self.gcn else 2 * self.feat_dim)
        )
        init.xavier_uniform(self.weight)
        
    def forward(self, nodes):
        # Generates embeddings for a batch of nodes. | nodes     -- list of nodes
        
        neigh_feats = self.aggregator.forward(nodes, [self.adj_lists[int(node)] for node in nodes], self.num_sample)
        if not self.gcn:
            if self.cuda:
                self_feats = self.features(torch.LongTensor(nodes).cuda())
            else:
                self_feats = self.features(torch.LongTensor(nodes))
            combined = torch.cat([self_feats, neigh_feats], dim=1)
        else:
            combined = neigh_feats
            
        combined = F.relu(self.weight.mm(combined.t()))
        
        return combined

### MeanAggregator
- Set of modules for aggregating embeddings of neighbors.

In [24]:
class MeanAggregator(nn.Module):
    # Aggregates a node's embeddings using mean of neighbors' embeddings
    def __init__(self, features, cuda=False, gcn=False):
        # Initializes the aggregator for a specific graph.
        # features -- function mapping LongTensor of node ids to FloatTensor of feature values.
        # cuda -- whether to use GPU
        # gcn --- whether to perform concatenation GraphSAGE-style, or add self-loops GCN-style
        super(MeanAggregator, self).__init__()
        
        self.features = features
        self.cuda = cuda
        self.gcn = gcn
        
    def forward(self, nodes, to_neighs, num_sample=10):
        """
        nodes --- list of nodes in a batch
        to_neighs --- list of sets, each set is the set of neighbors for node in batch
        num_sample --- number of neighbors to sample. No sampling if None.
        """
        # Local pointers to functions (speed hack)
        
        _set = set
        if not num_sample is None:
            _sample = random.sample
            samp_neighs = [_set(
                _sample(to_neigh, num_sample, )
            ) if len(to_neigh) >= num_sample else to_neigh for to_neigh in to_neighs]
            
        else:
            samp_neighs = to_neighs
            
        if self.gcn:
            samp_neighs = [samp_neigh + set([nodes[i]]) for i, samp_neigh in enumerate(samp_neighs)]
            
        unique_nodes_list = list(set.union(*samp_neighs))
        unique_nodes = {n: i for i, n in enumerate(unique_nodes_list)}
        mask = Variable(torch.zeros(len(samp_neighs), len(unique_nodes)))
        column_indices = [unique_nodes[n] for samp_neigh in samp_neighs for n in samp_neigh]   
        row_indices = [i for i in range(len(samp_neighs)) for j in range(len(samp_neighs[i]))]
        mask[row_indices, column_indices] = 1
        
        if self.cuda:
            mask = mask.cuda()
        num_neigh = mask.sum(1, keepdim=True)
        mask = mask.div(num_neigh)
        
        if self.cuda:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list).cuda())
        else:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list))
        to_feats = mask.mm(embed_matrix)
        
        return to_feats

In [25]:
class SupervisedGraphSage(nn.Module):
    def __init__(self, num_classes, enc):
        super(SupervisedGraphSage, self).__init__()
        self.enc = enc
        self.xent = nn.CrossEntropyLoss()
        
        self.weight = nn.Parameter(torch.FloatTensor(num_classes, enc.embed_dim))
        init.xavier_uniform(self.weight)
        
    def forward(self, nodes):
        embeds = self.enc(nodes)
        scores = self.weight.mm(embeds)
        return scores.t()
    
    def loss(self, nodes, labels):
        scores = self.forward(nodes)
        return self.xent(scores, labels.squeeze())

### 训练

In [30]:
def run_cora():
    np.random.seed(1)
    random.seed(1)
    num_nodes = 2708
    feat_data, labels, adj_lists = load_cora()
    features = nn.Embedding(2708, 1433)
    features.weight = nn.Parameter(torch.FloatTensor(feat_data), requires_grad=False)  # weight参数
    # feature.cuda()
    
    agg1 = MeanAggregator(features, cuda=True)
    enc1 = Encoder(features, 1433, 128, adj_lists, agg1, gcn=True, cuda=False)
    agg2 = MeanAggregator(lambda nodes : enc1(nodes).t(), cuda=False)
    enc2 = Encoder(lambda nodes: enc1(nodes).t(), enc1.embed_dim, 128, adj_lists, agg2, base_model=enc1, gcn=True, cuda=False)
    enc1.num_samples = 5
    enc2.num_samples = 5
    
    graphsage = SupervisedGraphSage(7, enc2)
    # graphsage.cuda()
    rand_indices = np.random.permutation(num_nodes)
    test = rand_indices[: 1000]
    val = rand_indices[1000: 1500]
    train = list(rand_indices[1500:])
    
    optimizer = torch.optim.SGD(filter(lambda p : p.requires_grad, graphsage.parameters()), lr=0.7)
    
    times = []
    for batch in range(1, 101):
        batch_nodes = train[: 256]
        random.shuffle(train)
        start_time = time.time()
        optimizer.zero_grad()
        loss = graphsage.loss(batch_nodes, Variable(torch.LongTensor(labels[np.array(batch_nodes)])))
        loss.backward()
        optimizer.step()
        end_time = time.time()
        times.append(end_time - start_time)
        if batch % 10 == 0:
            print(batch, loss.item())
    
    val_output = graphsage.forward(val)
    print("Validation F1:", f1_score(labels[val], val_output.data.numpy().argmax(axis=1), average="micro"))
    print("Average batch time:", np.mean(times))

In [31]:
run_cora()

c:\users\sss\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
c:\users\sss\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  


10 1.618682622909546
20 1.1853821277618408
30 0.8934508562088013
40 0.4756213426589966
50 0.38028547167778015
60 0.2593066394329071
70 0.28790998458862305
80 0.23371784389019012
90 0.23076742887496948
100 0.20929664373397827
Validation F1: 0.868
Average batch time: 0.06801058292388916


In [33]:
def run_pubmed():
    np.random.seed(1)
    random.seed(1)
    num_nodes = 19717
    feat_data, labels, adj_lists = load_pubmed()
    features = nn.Embedding(19717, 500)
    features.weight = nn.Parameter(torch.FloatTensor(feat_data), requires_grad=False)
    # features.cuda()
    
    agg1 = MeanAggregator(features, cuda=True)
    enc1 = Encoder(features, 500, 128, adj_lists, agg1, gcn=True, cuda=False)
    agg2 = MeanAggregator(lambda nodes : enc1(nodes).t(), cuda=False)
    enc2 = Encoder(lambda nodes : enc1(nodes).t(), enc1.embed_dim, 128, adj_lists, agg2, base_model=enc1, gcn=True, cuda=False)
    enc1.num_samples = 10
    enc2.num_samples = 25
    
    graphsage = SupervisedGraphSage(3, enc2)
    # graphsage.cuda()
    rand_indices = np.random.permutation(num_nodes)
    test = rand_indices[: 1000]
    val = rand_indices[1000: 1500]
    train = list(rand_indices[1500:])
    
    optimizer = torch.optim.SGD(filter(lambda p : p.requires_grad, graphsage.parameters()), lr=0.7)
    times = []
    for batch in range(1, 101):
        batch_nodes = train[: 1024]
        random.shuffle(train)
        start_time = time.time()
        optimizer.zero_grad()
        loss = graphsage.loss(batch_nodes, Variable(torch.LongTensor(labels[np.array(batch_nodes)])))
        loss.backward()
        optimizer.step()
        end_time = time.time()
        times.append(end_time - start_time)
        if batch % 10 == 0:
            print(batch, loss.item())
            
    val_output = graphsage.forward(val)
    print("Validation F1:", f1_score(labels[val], val_output.data.numpy().argmax(axis=1), average="micro"))
    print("Average batch time:", np.mean(times))    

In [34]:
run_pubmed()

c:\users\sss\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
c:\users\sss\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  


10 1.0965441465377808
20 1.0941740274429321
30 1.0901002883911133
40 1.0866940021514893
50 1.0807478427886963
60 1.0695284605026245
70 1.0590918064117432
80 1.0319048166275024
90 0.9996737837791443
100 0.9550232887268066
Validation F1: 0.514
Average batch time: 0.45042718172073365
